In [7]:
# ACTION Reduce by 10%, 20% etc -> increase action space
import argparse
import os
import random
import time
from distutils.util import strtobool

import gymnasium as gym
from gym.spaces import Discrete, Box
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

possible_actions = ["increase by level 1","increase by level 2","increase by level 3","maintain",
           "decrease by level 1","decrease by level 2","decrease by level 3"]

action_for_each_state = ()

In [59]:
# DOUBTS: Should co2 be considered a state space
# state_space1 = (gamma, pue, tdp_watts, config, chips) 
# state_space2 = (gamma1, pue1, tdp_watts1, config1, chips1) 
# difference = ce.state1 - ce.state2 
# difference is positive -> + reward
# difference is negative -> - reward  



# goal -> 395-405 -> terminal state reached 
# budget limit = 100
# current e = 500

# diff = 400



# Environment Implementation 
class env():
    # start from state and then take an action to return next state and the reward in the next state
    def __init__(self, curr_state,termination_co2):
        # 7 actions can be taken 
        self.action_space = Discrete(7)  
        # set start state
        
        self.curr_state = curr_state
        self.termination_co2 = termination_co2
        self.info_action = (0,0,0,0,0)
        super().__init__()

    
    def carbon_emissions(self,curr_state):
        product = 1
        # state_space1 = (gamma, pue, tdp_watts, config, chips) 
        for i in range(len(curr_state)):
            product *= curr_state[i]
        return product
    
       
    def step(self,curr_state):
        # state_space1 = (gamma, pue, tdp_watts, config, chips) 
        # info_action = (0,4,5,6,3)
        reward = 0
        prev_co2 = self.carbon_emissions(curr_state)
        
        # each state component takes an action 
        state_list = list(curr_state)
        actions_list = list(self.info_action)
        
        for i in range(len(state_list)):
            state_list[i] += actions_list[i] 
            
        self.curr_state = tuple(state_list)
        
        new_state = curr_state
        
        new_co2 = self.carbon_emissions(new_state)
        
        old_diff = abs(self.termination_co2 - prev_co2)
        new_diff = abs(self.termination_co2 - new_co2)
        
        if new_diff <= 0.1*self.termination_co2:
            done = True
        else:
            # 100 - 500 = 400 -> old diff
            # 100 - 200 = 200 -> new diff 
            # 200 - 400  = - 200 
            if new_diff - old_diff >0: 
                reward -=1
            else:
                reward += 1
            done = False
        # info could be actions_list ?????????
        info = {}
        return self.curr_state, reward, done, info
        
    # difference between reset and init
    def reset(self, curr_state,termination_co2):
        self.action_space = Discrete(7)   
        
        self.curr_state = curr_state
        
        self.termination_co2 = termination_co2
        
        self.info_action = (0,0,0,0,0)
        
        return curr_state


In [60]:
state = (1,2,3,4,5)
env1 = env(state,100)
prod = env1.step(state)
print(prod)

((1, 2, 3, 4, 5), 1, False, {})


In [ ]:
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(np.array(5).prod(), 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 3),
        )
    # x is state
    def forward(self, x):
        return self.network(x)


In [2]:
import numpy as np

# Define the state space
states = ['gamma', 'pue', 'grid_config','tdp_watts','chips_num',"co2_emissions"]
state_space = [0,1,2,3,4]  # representing low, medium, high expenses for each category

# Define the action space - will it mention the extent of change???? gamma, pue, tdp_watts needs to be more specific 
actions = ['reduce by 1 level', 'maintain', 'increase']    # elaborate

#numeric action codes: 0 = reduce, 1 = maintain, 2 = increase
action_space = [0,1,2]  # representing reducing, maintaining, or increasing expenses for each category



# # 40 * 8 * 221 = 320*221 = 70,720 * 20 * 100 = 141,440,000




# Doubt:    Reward for the goal????

# initial state: The current co2 emissions of the model
# terminal state: co2 emission reduced to the given budget (+-5%)

goal_state = 100 
initial_state = 500 


# self.network = nn.Sequential(
#     nn.Linear(np.array(env.single_observation_space.shape).prod(), 120),
#     nn.ReLU(),
#     nn.Linear(120, 84),
#     nn.ReLU(),
#     nn.Linear(84, env.single_action_space.n),
# )


In [3]:
# docs and experiment results can be found at https://docs.cleanrl.dev/rl-algorithms/dqn/#dqnpy
import argparse
import os
import random
import time
from distutils.util import strtobool

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

# ALGO LOGIC: initialize agent here:
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(np.array(5).prod(), 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 3),
        )
    # x is state
    def forward(self, x):
        return self.network(x)


In [ ]:
# python dqn.py --total-timesteps 500000 \ 500 
#     --learning-rate 2.5e-4 \
#     --buffer-size 10000 \
#     --gamma 0.99 \
#     --target-network-frequency 500 \ 0.5
#     --max-grad-norm 0.5 \
#     --batch-size 128 \
#     --start-e 1 \
#     --end-e 0.05 \
#     --exploration-fraction 0.5 \
#     --learning-starts 10000 ???// 10
#     --train-frequency 10 ????? 2 

In [15]:
class env():
    # start from state and then take an action to return next state and the reward in the next state
    def __init__(self):
        super().__init__()

    def reset(self):
        return 
    def step(self, action):
        return reward_in_next_state, next_state


In [5]:
    device = torch.device("cuda" if torch.cuda.is_available() and args.cuda else "cpu")
    q_network = QNetwork().to(device)
    optimizer = optim.Adam(q_network.parameters(), lr=2.5e-4)
    target_network = QNetwork().to(device)
    
    # resume the training -> checkpoint -> starts from checkpoint
    target_network.load_state_dict(q_network.state_dict())

<All keys matched successfully>

In [ ]:
 # Seed -> random action probability to encourage exploration -> reproducablility -> controls the randomness
obs, _ = envs.reset(seed=args.seed)
    for global_step in range(500):
        # ALGO LOGIC: put action logic here
        epsilon = linear_schedule(1, 0.05, 0.5 * 500, global_step)
        if random.random() < epsilon:
            # actions space in an array (np.array)
            actions = np.array([actions for _ in range(3)])
        else:
            q_values = q_network(torch.Tensor(obs).to(device))
            actions = torch.argmax(q_values, dim=1).cpu().numpy()

        # TRY NOT TO MODIFY: execute the game and log data.
        # next_observation -> how to initialise observations
        # rewards: where do we initalise rewards
        # truncated, infos came from 
        # terminated: +-5% of the desired budget, so how do we formulate that in the code
        next_obs, rewards, terminated, truncated, infos = envs.step(actions)

#         # TRY NOT TO MODIFY: record rewards for plotting purposes
#         if "final_info" in infos:
#             for info in infos["final_info"]:
#                 # Skip the envs that are not done
#                 if "episode" not in info:
#                     continue
#                 print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
#                 writer.add_scalar("charts/episodic_return", info["episode"]["r"], global_step)
#                 writer.add_scalar("charts/episodic_length", info["episode"]["l"], global_step)
#                 writer.add_scalar("charts/epsilon", epsilon, global_step)




# Doubts ????? DO WE REALLY NEED BUFFER?
#         # TRY NOT TO MODIFY: save data to reply buffer; handle `final_observation`
#         real_next_obs = next_obs.copy()
#         for idx, d in enumerate(truncated):
#             if d:
#                 real_next_obs[idx] = infos["final_observation"][idx]
#         rb.add(obs, real_next_obs, actions, rewards, terminated, infos)

#         # TRY NOT TO MODIFY: CRUCIAL step easy to overlook
#         obs = next_obs

        # ALGO LOGIC: training.
        if global_step > 10:
            if global_step % 2 == 0:
                #Doubt: Buffer rb. sample, batch size??????
                data = rb.sample(args.batch_size)
                
                
                # DOUBTS: What is this code doing??????
                with torch.no_grad():
                    target_max, _ = target_network(data.next_observations).max(dim=1)
                    td_target = data.rewards.flatten() + 0.99 * target_max * (1 - data.dones.flatten())
                old_val = q_network(data.observations).gather(1, data.actions).squeeze()
                loss = F.mse_loss(td_target, old_val)

#                 if global_step % 100 == 0:
#                     writer.add_scalar("losses/td_loss", loss, global_step)
#                     writer.add_scalar("losses/q_values", old_val.mean().item(), global_step)
#                     print("SPS:", int(global_step / (time.time() - start_time)))
#                     writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

                # optimize the model
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # update target network
            
            # DOUBT: What is tau - kept it as 1 for now
            if global_step % 0.5 == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        args.tau * q_network_param.data + (1.0 - args.tau) * target_network_param.data
                    )
